In [4]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\newvision\appdata\local\temp\pip-req-build-lpa54qxv
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\NewVision\AppData\Local\Temp\pip-req-build-lpa54qxv'


In [5]:
!git clone https://github.com/timesler/facenet-pytorch.git facenet_pytorch

fatal: destination path 'facenet_pytorch' already exists and is not an empty directory.


In [7]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import uvicorn
from PIL import Image
import torch
import open_clip
import cv2
from sentence_transformers import util
from facenet_pytorch import MTCNN, InceptionResnetV1
import io
import logging

In [4]:
# Initialize FastAPI
app = FastAPI()

In [9]:
# Load the models
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model.to(device)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 896, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((896,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((896,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=896, out_features=896, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((896,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=896, out_features=3584, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3584, out_features=896, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((896,), eps=1e-05, elementwise_affine

In [10]:
resnet = InceptionResnetV1(pretrained='vggface2').eval()


In [11]:
def imageEncoder(img):
    if img.dtype != 'uint8':
        img = img.astype('uint8')  # Convert to 8-bit if necessary
    img1 = Image.fromarray(img).convert('RGB')
    img1 = preprocess(img1).unsqueeze(0).to(device)
    img1 = model.encode_image(img1)
    return img1

In [15]:
def crop_comp_img(person, person_id):
    
    person_crop_path = person.split(".")[0] + "_crop.jpg"
    person_id_crop_path = person_id.split(".")[0] + "_crop.jpg"
    
    # If required, create a face detection pipeline using MTCNN:
    mtcnn = MTCNN(image_size=160, margin=0)

    # Create an inception resnet (in eval mode):
    resnet = InceptionResnetV1(pretrained='vggface2').eval()


    person_img = Image.open(person)
    person_id_img = Image.open(person_id)

     # Get cropped and prewhitened image tensor
    person_crop = mtcnn(person_img,save_path = person_crop_path)
    person_id_crop = mtcnn(person_id_img, save_path = person_id_crop_path)
    
    # Calculate embedding (unsqueeze to add batch dimension)
    person_crop_embedding = resnet(person_crop.unsqueeze(0))
    person_id_crop_embedding = resnet(person_id_crop.unsqueeze(0))
    
    # Or, if using for VGGFace2 classification
    resnet.classify = True

    person_crop_probs = resnet(person_crop.unsqueeze(0))
    person_id_crop_probs= resnet(person_id_crop.unsqueeze(0))


    person_compare = cv2.imread(person_crop_path, cv2.IMREAD_UNCHANGED)
    person_id_compare = cv2.imread(person_id_crop_path, cv2.IMREAD_UNCHANGED)
    img1 = imageEncoder(person_compare)
    img2 = imageEncoder(person_id_compare)
    cos_scores = util.pytorch_cos_sim(img1, img2)
    score = round(float(cos_scores[0][0]) * 100, 2)
    return score, person_compare, person_id_compare

In [16]:
@app.post("/compare/")
async def compare_images(file1: UploadFile = File(...), file2: UploadFile = File(...)):
    try:
        # Read uploaded files
        image1 = Image.open(io.BytesIO(await file1.read()))
        image2 = Image.open(io.BytesIO(await file2.read()))

        # Generate score
        score = crop_comp_img(image1, image2)
        result = "Same Person" if score >= 60 else "Not the same Person"
        return JSONResponse(content={"score": score, "result": result})
    except Exception as e:
        logging.error(f"Error in compare_images endpoint: {e}")
        raise HTTPException(status_code=500, detail=f"Internal Server Error: {e}")

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


NameError: name 'app' is not defined

In [13]:
bahaa = "C:\\Users\\NewVision\\Downloads\\image_model\\bahaa.jpg"
bahaa_id = "C:\\Users\\NewVision\\Downloads\\image_model\\front_id.jpg"

In [14]:
crop_comp_img(bahaa,bahaa_id)

AttributeError: 'str' object has no attribute 'size'